In [51]:
from qiskit import QuantumCircuit, Aer, IBMQ, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error # type: ignore

# Check if IBMQ account is already loaded, otherwise, load it with the token
if IBMQ.active_account() is None:
    IBMQ.save_account('03927d9258ef7fd9fbacf9dbee829b761fbf0f8beea3966880e3a8870bfec3badb1ff3d4425e42361746c5ed0ca9e207cb34890dcddbf7929846390b60ba5c45', overwrite=True)
    IBMQ.load_account()
else:
    print("IBM Quantum account already loaded.")


NameError: name 'IBMQ' is not defined

In [ ]:
qc = QuantumCircuit(4, 4)
qc.h(range(4))  # Apply Hadamard to all qubits
qc.cz(0, 1)  # Controlled-Z gate
qc.cx(1, 2)  # CNOT gate
qc.t(3)  # T gate on qubit 3
qc.s(2)  # S gate on qubit 2
qc.z(1)  # Pauli-Z on qubit 1
qc.measure(range(4), range(4))
print(qc)

ImportError: cannot import name 'Aer' from 'qiskit' (/Users/abdullahawan/coding_with_qiskit/miniconda3/envs/cwq/lib/python3.12/site-packages/qiskit/__init__.py)

In [ ]:
noise_model = NoiseModel()
error = depolarizing_error(0.02, 1)
noise_model.add_all_qubit_quantum_error(error, ['h', 'cx', 'cz', 't', 's', 'z'])

IBMNotAuthorizedError: '401 Client Error: Unauthorized for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Login failed., Error code: 3446.'

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
result = execute(qc, simulator, noise_model=noise_model).result()
counts = result.get_counts(qc)
plot_histogram(counts)

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
real_device = provider.get_backend('ibmq_manila')
job = execute(qc, backend=real_device)
real_counts = job.result().get_counts()
plot_histogram([counts, real_counts], legend=['Simulated', 'Real'])
